In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tar/fiuba_4_postulaciones.csv')
avisosOnline = pd.read_csv('resources/tar/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('resources/tar/fiuba_6_avisos_detalle.csv')

In [2]:
testingSet = pd.read_csv('resources/test_final_100k.csv')

# Reducimos a algo minimo para probar algoritmos

In [3]:
postulacionesSample = postulaciones.sample(frac=0.00001)

In [12]:
postulacionesSample.info

<bound method DataFrame.info of             idaviso idpostulante     fechapostulacion
548054   1112285953      96XzlN8  2018-02-01 05:19:20
2643033  1112369058      owj6dPk  2018-03-09 08:31:25
3562866  1112448632      BmBxmjR  2018-04-06 20:07:15
4648591  1111930643      qevrEJb  2018-04-08 11:55:08
2862330  1112368514      8MBXGNx  2018-03-11 10:27:35
4744422  1112266469      3NlpAkx  2018-03-27 08:33:17
1229441  1111972061      kPNwRQV  2018-01-29 10:48:20
1675671  1112372532      BmaZz1R  2018-03-01 20:00:07
1667449  1112269370       64lQDl  2018-01-22 18:14:03
706997   1112458518      0zB0YGv  2018-04-11 18:44:10
952941   1112252543      KBrG5oA  2018-01-19 03:32:18
4593251  1112312923      W9A1PAe  2018-02-14 09:29:07
188143   1112403375      GNZq2YK  2018-03-15 18:45:51
3965957  1112313265       5wP1vw  2018-02-07 02:51:52
2747291  1112170115      3NQmDoJ  2018-03-15 12:07:58
4038301  1112418593       eApZ12  2018-03-22 22:15:42
668599   1112396657      A3XP5El  2018-03-13 19:11

# Categoricals a Numericals

## Postulantes Genero

In [38]:
import datetime as dt

postulantesGenero['fechanacimiento'] = pd.to_datetime(postulantesGenero['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
postulantesGenero = postulantesGenero.dropna(subset=['fechanacimiento'])

hoy = pd.Timestamp(dt.datetime.now())
postulantesGenero['edad'] = (hoy - postulantesGenero['fechanacimiento']).astype('<m8[Y]')

In [39]:
postulantesGeneroOHC = pd.concat([postulantesGenero,pd.get_dummies(postulantesGenero['sexo'], prefix='sexo')],axis=1)

postulantesGeneroOHC = postulantesGeneroOHC[postulantesGeneroOHC['sexo_0.0'] == 0]
postulantesGeneroOHC = postulantesGeneroOHC[['idpostulante','fechanacimiento','sexo_FEM','sexo_MASC','sexo_NO_DECLARA','edad']]

In [40]:
postulantesGeneroOHC.head()

,idpostulante,fechanacimiento,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,edad
0,6MM,1985-01-01,0,1,0,33.0
4,ebE,1952-07-07,0,1,0,65.0
7,NAjM,1962-06-09,1,0,0,56.0
10,ZjlZ,1970-01-25,0,1,0,48.0
11,8wPl,1973-03-13,0,1,0,45.0


## Educacion 

In [35]:
postulantesEducacionOHC = pd.concat([postulantesEducacion,pd.get_dummies(postulantesEducacion['estado'], prefix='estado')],axis=1)
postulantesEducacionOHC = pd.concat([postulantesEducacionOHC,pd.get_dummies(postulantesEducacion['nombre'], prefix='nombre')],axis=1)

postulantesEducacionOHC.drop(['nombre'],axis=1, inplace=True)
postulantesEducacionOHC.drop(['estado'],axis=1, inplace=True)

## Detalle 

In [5]:
datasetParaMerge = postulacionesSample;

In [7]:
#Me quedo con los avisos que solo se postulo para persona para simplificar


In [9]:
avisosDetalleFiltered = avisosDetalle.loc[avisosDetalle['nombre_zona'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nombre_area'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['tipo_de_trabajo'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nivel_laboral'].notnull(),:]

In [10]:
avisosDetalleFiltered['denominacion_empresa'].fillna('S/E', inplace=True)

In [12]:
avisosPostulados = pd.merge(datasetParaMerge,avisosDetalleFiltered, on='idaviso',how='inner');

In [24]:
avisosParaOHC = avisosPostulados;

In [25]:
avisosDetalleOHC = pd.concat([avisosParaOHC,pd.get_dummies(avisosParaOHC['nombre_zona'], prefix='zona')],axis=1)
avisosDetalleOHC = pd.concat([avisosParaOHC,pd.get_dummies(avisosParaOHC['tipo_de_trabajo'], prefix='tipo')],axis=1)
avisosDetalleOHC = pd.concat([avisosParaOHC,pd.get_dummies(avisosParaOHC['nivel_laboral'], prefix='nivel')],axis=1)
avisosDetalleOHC = pd.concat([avisosParaOHC,pd.get_dummies(avisosParaOHC['nombre_area'], prefix='area')],axis=1)

In [26]:
avisosDetalleOHC.drop(['ciudad','mapacalle'],axis=1,inplace=True)

In [16]:
avisosDetalleOHC.head()

,idaviso,idpostulante,fechapostulacion,idpais,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,...,area_Marketing,area_Otros,area_Producción,area_Programación,area_Recepcionista,area_Recursos Humanos,area_Redes,area_Sistemas,area_Telemarketing,area_Ventas
0,1112431297,8MPe1pl,2018-03-28 17:31:56,1,Telemarketer con experiencia - EXCLUYENTE TURN...,<p>¿Buscas un trabajo de 6 horas? ¿Queres trab...,Gran Buenos Aires,Part-time,Senior / Semi-Senior,Ventas,...,0,0,0,0,0,0,0,0,0,1
1,1112264902,0zr6BD8,2018-01-18 06:50:25,1,Pasante de Marketing,"<p style=""""><strong><span style="""">¿Te gustarí...",Gran Buenos Aires,Pasantia,Junior,Marketing,...,1,0,0,0,0,0,0,0,0,0
2,1112172032,3NPGJwP,2018-01-24 14:22:21,1,Analista contable - Eventual por licencia,<p><strong>Adecco Office</strong> está especia...,Gran Buenos Aires,Full-time,Junior,Contabilidad,...,0,0,0,0,0,0,0,0,0,0
3,1112406733,0zrdBMv,2018-03-17 11:26:49,1,Vendedor telefónico PART-TIME TURNO TARDE Call...,"<p>Activo Contact Center, empresa ubicada en e...",Gran Buenos Aires,Part-time,Senior / Semi-Senior,Telemarketing,...,0,0,0,0,0,0,0,0,1,0
4,1112347174,rm2Rwv1,2018-03-19 09:31:51,1,Auxiliar de Experiencia al Cliente Aeroparque,"<p><span style="""">Nos encontramos en la búsque...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Atención al Cliente,...,0,0,0,0,0,0,0,0,0,0


# Convierto Empresas a Numerical

In [27]:
postulacionesConAvisos = pd.merge(avisosDetalleOHC,postulaciones, how='inner',on='idaviso')

In [28]:
postulacionesConAvisos['cantidadEmpresa'] = 1;
cantAvisosPorEmpresa = postulacionesConAvisos.groupby('denominacion_empresa').sum()
cantAvisosPorEmpresa = cantAvisosPorEmpresa.reset_index()
cantAvisosPorEmpresa = cantAvisosPorEmpresa[['denominacion_empresa','cantidadEmpresa']]

In [124]:
cantAvisosPorEmpresa.head()

,denominacion_empresa,cantidadEmpresa
0,AB InBev - Cervecería y Maltería Quilmes,1
1,ABAC S R L,1
2,ALESSANDRINI S R L,1
3,Adecco - Región NORTE & OESTE GBA,1
4,Adecco -Región GBA SUR,1


In [29]:
avisosConEmpresaNumericals = pd.merge(avisosDetalleOHC,cantAvisosPorEmpresa,how='inner',on='denominacion_empresa')

In [30]:
avisosConEmpresaNumericals.drop('denominacion_empresa',inplace=True,axis=1)
#Considero a la empresa como cantidad, esto agruparia tambien a las que son iguales

In [31]:
avisosConEmpresaNumericals.head()

,idaviso,idpostulante,fechapostulacion,idpais,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,...,area_Otros,area_Producción,area_Programación,area_Recepcionista,area_Recursos Humanos,area_Redes,area_Sistemas,area_Telemarketing,area_Ventas,cantidadEmpresa
0,1112431297,8MPe1pl,2018-03-28 17:31:56,1,Telemarketer con experiencia - EXCLUYENTE TURN...,<p>¿Buscas un trabajo de 6 horas? ¿Queres trab...,Gran Buenos Aires,Part-time,Senior / Semi-Senior,Ventas,...,0,0,0,0,0,0,0,0,1,2386
1,1112315655,Ejb554,2018-02-06 21:07:27,1,Operador telefónico para ventas de seguros - T...,"<p>Esta es tu oportunidad, para que puedas inc...",Gran Buenos Aires,Part-time,Otro,Call Center,...,0,0,0,0,0,0,0,0,0,2386
2,1112264902,0zr6BD8,2018-01-18 06:50:25,1,Pasante de Marketing,"<p style=""""><strong><span style="""">¿Te gustarí...",Gran Buenos Aires,Pasantia,Junior,Marketing,...,0,0,0,0,0,0,0,0,0,1050
3,1112172032,3NPGJwP,2018-01-24 14:22:21,1,Analista contable - Eventual por licencia,<p><strong>Adecco Office</strong> está especia...,Gran Buenos Aires,Full-time,Junior,Contabilidad,...,0,0,0,0,0,0,0,0,0,230
4,1112406733,0zrdBMv,2018-03-17 11:26:49,1,Vendedor telefónico PART-TIME TURNO TARDE Call...,"<p>Activo Contact Center, empresa ubicada en e...",Gran Buenos Aires,Part-time,Senior / Semi-Senior,Telemarketing,...,0,0,0,0,0,0,0,1,0,2312


In [32]:
avisosConEmpresaNumericals.drop(['nombre_zona'],axis=1, inplace=True)
avisosConEmpresaNumericals.drop(['tipo_de_trabajo'],axis=1, inplace=True)
avisosConEmpresaNumericals.drop(['nivel_laboral'],axis=1, inplace=True)
avisosConEmpresaNumericals.drop(['nombre_area'],axis=1, inplace=True)

In [33]:
avisosConEmpresaNumericals.head()

,idaviso,idpostulante,fechapostulacion,idpais,titulo,descripcion,area_Administración,area_Administración de Personal,area_Apoderado Aduanal,area_Atención al Cliente,...,area_Otros,area_Producción,area_Programación,area_Recepcionista,area_Recursos Humanos,area_Redes,area_Sistemas,area_Telemarketing,area_Ventas,cantidadEmpresa
0,1112431297,8MPe1pl,2018-03-28 17:31:56,1,Telemarketer con experiencia - EXCLUYENTE TURN...,<p>¿Buscas un trabajo de 6 horas? ¿Queres trab...,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2386
1,1112315655,Ejb554,2018-02-06 21:07:27,1,Operador telefónico para ventas de seguros - T...,"<p>Esta es tu oportunidad, para que puedas inc...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,2386
2,1112264902,0zr6BD8,2018-01-18 06:50:25,1,Pasante de Marketing,"<p style=""""><strong><span style="""">¿Te gustarí...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,1050
3,1112172032,3NPGJwP,2018-01-24 14:22:21,1,Analista contable - Eventual por licencia,<p><strong>Adecco Office</strong> está especia...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,230
4,1112406733,0zrdBMv,2018-03-17 11:26:49,1,Vendedor telefónico PART-TIME TURNO TARDE Call...,"<p>Activo Contact Center, empresa ubicada en e...",0,0,0,0,...,0,0,0,0,0,0,0,1,0,2312


# Mergeamos todo

In [41]:
#todoMergeado = pd.merge(postulaciones,avisosDetalleOHC, how='inner',on='idaviso');
todoMergeado = avisosConEmpresaNumericals;
todoMergeado = pd.merge(postulantesEducacionOHC,todoMergeado, how='inner',on='idpostulante');
todoMergeado = pd.merge(postulantesGeneroOHC,todoMergeado, how='inner',on='idpostulante');

In [42]:
todoMergeado['sepostulo'] = 1;

In [44]:
todoMergeadoFiltrado = todoMergeado.drop(['fechanacimiento'],axis=1)
todoMergeadoFiltrado = todoMergeadoFiltrado.drop(['descripcion'],axis=1)
todoMergeadoFiltrado = todoMergeadoFiltrado.drop(['titulo'],axis=1)

In [46]:
todoMergeadoFiltrado.head()

,idpostulante,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,edad,estado_Abandonado,estado_En Curso,estado_Graduado,nombre_Doctorado,nombre_Master,...,area_Producción,area_Programación,area_Recepcionista,area_Recursos Humanos,area_Redes,area_Sistemas,area_Telemarketing,area_Ventas,cantidadEmpresa,sepostulo
0,8rk0MR,1,0,0,36.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,683,1
1,ElQv8z,1,0,0,45.0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,109,1
2,ZRdbNY,0,1,0,32.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1772,1
3,6abbmM,0,1,0,41.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,339,1
4,EBzrLz,1,0,0,33.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,12539,1


## Creamos avisos no postulados 

In [86]:
datasetPruebas = postulacionesSample;

In [87]:
avisosVistos = pd.merge(datasetPruebas,vistas,on='idpostulante',how='inner')

In [88]:
avisosVistos.drop('fechapostulacion',axis=1,inplace=True)

In [69]:
avisosVistos.head()

,idaviso,idpostulante,idAviso,timestamp
0,1112431297,8MPe1pl,1112417350,2018-03-22T10:23:57.501-0400
1,1112431297,8MPe1pl,1112417350,2018-03-22T10:23:55.315-0400
2,1112431297,8MPe1pl,1112417350,2018-03-22T10:26:33.860-0400
3,1112431297,8MPe1pl,1112417348,2018-03-22T10:29:01.612-0400
4,1112431297,8MPe1pl,1112417350,2018-03-22T10:26:36.178-0400


In [89]:
avisosVistosNoPostulados = pd.merge(datasetPruebas,avisosVistos,left_on=['idpostulante','idaviso'],right_on=['idpostulante','idAviso'],how='right')

In [72]:
avisosVistosNoPostulados.head()

,idaviso_x,idpostulante,fechapostulacion,idaviso_y,idAviso,timestamp
0,1.112423e+09,N90rxV,2018-03-28 17:47:03,1112423441,1112423441,2018-03-28T17:46:40.404-0400
1,1.112423e+09,N90rxV,2018-03-28 17:47:03,1112423441,1112423441,2018-03-28T17:46:41.914-0400
2,1.112445e+09,6rBLEEl,2018-04-05 20:15:11,1112444567,1112444567,2018-04-05T20:14:39.763-0400
3,1.112287e+09,akjmL09,2018-02-16 18:12:11,1112286978,1112286978,2018-03-28T13:27:55.214-0400
4,1.112287e+09,akjmL09,2018-02-16 18:12:11,1112286978,1112286978,2018-03-28T13:27:59.634-0400


In [90]:
avisosVistosNoPostuladosFinal = avisosVistosNoPostulados[avisosVistosNoPostulados['fechapostulacion'].isnull()]

In [74]:
avisosVistosNoPostuladosFinal.head(30)

,idaviso_x,idpostulante,fechapostulacion,idaviso_y,idAviso,timestamp
19,NaN,8MPe1pl,NaN,1112431297,1112417350,2018-03-22T10:23:57.501-0400
20,NaN,8MPe1pl,NaN,1112431297,1112417350,2018-03-22T10:23:55.315-0400
21,NaN,8MPe1pl,NaN,1112431297,1112417350,2018-03-22T10:26:33.860-0400
22,NaN,8MPe1pl,NaN,1112431297,1112417350,2018-03-22T10:26:36.178-0400
23,NaN,8MPe1pl,NaN,1112431297,1112417350,2018-03-22T10:24:50.412-0400
24,NaN,8MPe1pl,NaN,1112431297,1112417350,2018-03-22T10:28:48.694-0400
25,NaN,8MPe1pl,NaN,1112431297,1112417348,2018-03-22T10:29:01.612-0400
26,NaN,8MPe1pl,NaN,1112431297,1112417341,2018-03-22T10:32:09.223-0400
27,NaN,8MPe1pl,NaN,1112431297,1112417341,2018-03-22T10:32:10.990-0400
28,NaN,8MPe1pl,NaN,1112431297,1000139175,2018-03-22T10:22:17.109-0400


In [91]:
avisosVistosNoPostuladosFinal.drop(['idaviso_x','fechapostulacion','idaviso_y'],axis=1,inplace=True)

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
avisosVistosNoPostuladosFinal['sepostulo'] = 0;

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [93]:
avisosVistosNoPostuladosFinal['idaviso'] = avisosVistosNoPostuladosFinal['idAviso'];
avisosVistosNoPostuladosFinal.drop('idAviso',inplace=True,axis=1);


C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [94]:
avisosVistosNoPostuladosFinal.drop(['timestamp'],axis=1,inplace=True)

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [102]:
avisosVistosNoPostuladosFinal.head()

,idpostulante,sepostulo,idaviso
19,8MPe1pl,0,1112417350
20,8MPe1pl,0,1112417350
21,8MPe1pl,0,1112417350
22,8MPe1pl,0,1112417350
23,8MPe1pl,0,1112417350


In [112]:
avisosVistosNoPostuladosFinal.info

<bound method DataFrame.info of      idpostulante  sepostulo     idaviso
19        8MPe1pl          0  1112417350
20        8MPe1pl          0  1112417350
21        8MPe1pl          0  1112417350
22        8MPe1pl          0  1112417350
23        8MPe1pl          0  1112417350
24        8MPe1pl          0  1112417350
25        8MPe1pl          0  1112417348
26        8MPe1pl          0  1112417341
27        8MPe1pl          0  1112417341
28        8MPe1pl          0  1000139175
29        8MPe1pl          0  1112417488
30        8MPe1pl          0  1112417522
31        8MPe1pl          0  1112417522
32        8MPe1pl          0  1112417416
33        8MPe1pl          0  1112417416
34        8MPe1pl          0  1112305373
35        8MPe1pl          0  1112317620
36        8MPe1pl          0  1112417275
37        8MPe1pl          0  1111503098
38        8MPe1pl          0  1111503098
39        8MPe1pl          0  1112418025
40        8MPe1pl          0  1112418025
41        8MPe1pl        

## Obtenemos las columnas de Genero, Detalle y Educacion para los avisos no postulados

In [115]:
 avisosNOSample = avisosVistosNoPostuladosFinal.sample(frac=0.05);

In [117]:
avisosNOSample.info

<bound method DataFrame.info of      idpostulante  sepostulo     idaviso
593       KBzm0ZX          0  1112442465
1757       6abbmM          0  1112351906
4228       Ejb554          0  1112425713
3567       Ejb554          0  1002490173
4126       Ejb554          0  1112442799
4010       Ejb554          0  1112418855
3081       ZRdbNY          0  1112442248
3980       Ejb554          0  1112455069
2099      jkd3DjY          0  1112433489
200       3NPGJwP          0  1112413366
3874       Ejb554          0  1112383408
2348      GNJPR6A          0  1112403133
464       KBzm0ZX          0  1112445682
1836       6abbmM          0  1112426663
3319      1Q4P48j          0  1112419006
62        0zr6BD8          0  1112438899
3626       Ejb554          0  1112439130
1540      akjmL09          0  1111709950
1533      akjmL09          0  1112460611
3573       Ejb554          0  1112441814
3089       ZRdbNY          0  1112422951
1584      JBrNX43          0  1112424876
2810      JBxNeoO        

In [118]:
todoMergeadoNO = pd.merge(avisosNOSample,avisosDetalleFiltered, how='inner',on='idaviso');

In [125]:
todoMergeadoNO.head()

,idpostulante,sepostulo,idaviso,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,KBzm0ZX,0,1112442465,Asesor/a de Alianzas Corporativas,"<p align=""center""><span lang=""DE"">Para imporat...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Relaciones Institucionales/Publicas,Psicotécnicos Gabriela Ianiselli &amp; Asociados
1,6abbmM,0,1112351906,Analista Administrativo de Ventas,<p>Seleccionaremos para Importante Laboratorio...,Gran Buenos Aires,Full-time,Junior,Administración,RANDSTAD
2,Ejb554,0,1112425713,Cajera - Part Time PSA (CABA),"<p style=""""><strong><span style="""">¡Estamos bu...",Gran Buenos Aires,Part-time,Junior,Atención al Cliente,PSA
3,pzMQ43p,0,1112425713,Cajera - Part Time PSA (CABA),"<p style=""""><strong><span style="""">¡Estamos bu...",Gran Buenos Aires,Part-time,Junior,Atención al Cliente,PSA
4,Ejb554,0,1002490173,Administrativa/o Comercial,<p>Somos una empresa que elabora y distribuye ...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Administración,OTOWIL


In [126]:
todoMergeadoNO.drop(['idpais','ciudad','mapacalle'],axis=1,inplace=True);

ValueError: labels ['idpais' 'ciudad' 'mapacalle'] not contained in axis

In [127]:
avisosNO = pd.concat([todoMergeadoNO,pd.get_dummies(todoMergeadoNO['nombre_zona'], prefix='zona')],axis=1)
avisosNO = pd.concat([avisosNO,pd.get_dummies(avisosNO['tipo_de_trabajo'], prefix='tipo')],axis=1)
avisosNO = pd.concat([avisosNO,pd.get_dummies(avisosNO['nivel_laboral'], prefix='nivel')],axis=1)
avisosNO = pd.concat([avisosNO,pd.get_dummies(avisosNO['nombre_area'], prefix='area')],axis=1)

In [128]:
avisosNO.drop(['nombre_zona'],axis=1, inplace=True)
avisosNO.drop(['tipo_de_trabajo'],axis=1, inplace=True)
avisosNO.drop(['nivel_laboral'],axis=1, inplace=True)
avisosNO.drop(['nombre_area'],axis=1, inplace=True)

In [129]:
avisosNOConEmpresas = pd.merge(avisosNO,cantAvisosPorEmpresa,on='denominacion_empresa',how='inner');

In [130]:
avisosNOConEmpresas.drop('denominacion_empresa',axis=1,inplace=True);

In [131]:
avisosNOConEmpresas.columns

Index(['idpostulante', 'sepostulo', 'idaviso', 'titulo', 'descripcion',
       'zona_Capital Federal', 'zona_Gran Buenos Aires', 'tipo_Full-time',
       'tipo_Part-time', 'tipo_Temporario',
       'nivel_Gerencia / Alta Gerencia / Dirección',
       'nivel_Jefe / Supervisor / Responsable', 'nivel_Junior', 'nivel_Otro',
       'nivel_Senior / Semi-Senior', 'area_Abastecimiento',
       'area_Administración', 'area_Almacén / Depósito / Expedición',
       'area_Asistente', 'area_Atención al Cliente', 'area_Auditoría',
       'area_Bioquimica', 'area_Calidad', 'area_Call Center',
       'area_Capacitación', 'area_Comercial', 'area_Compensación y Planilla',
       'area_Compras', 'area_Compras Internacionales/Importación',
       'area_Comunicacion', 'area_Contabilidad', 'area_Control de Gestión',
       'area_Creatividad', 'area_Créditos y Cobranzas',
       'area_Desarrollo de Negocios', 'area_Dirección de Obra',
       'area_Distribución', 'area_Facturación', 'area_Farmacéutica',
     

In [134]:
#GENERO
mergeNO = pd.merge(avisosNOConEmpresas,postulantesGeneroOHC,on='idpostulante',how='inner');

In [135]:
mergeNOFinal = pd.merge(mergeNO,postulantesEducacionOHC,on='idpostulante',how='inner');

In [136]:
todoMergeadoFINAL = pd.concat([todoMergeadoFiltrado, mergeNOFinal], axis=0, ignore_index=True)

In [138]:
todoMergeadoFINAL.columns

Index(['area_Abastecimiento', 'area_Administración',
       'area_Administración de Personal',
       'area_Almacén / Depósito / Expedición', 'area_Apoderado Aduanal',
       'area_Asistente', 'area_Atención al Cliente', 'area_Auditoría',
       'area_Bioquimica', 'area_Calidad', 'area_Call Center',
       'area_Capacitación', 'area_Comercial', 'area_Compensación y Planilla',
       'area_Compras', 'area_Compras Internacionales/Importación',
       'area_Comunicacion', 'area_Contabilidad', 'area_Control de Gestión',
       'area_Creatividad', 'area_Créditos y Cobranzas',
       'area_Desarrollo de Negocios', 'area_Dirección de Obra',
       'area_Distribución', 'area_Facturación', 'area_Farmacéutica',
       'area_Finanzas', 'area_Gastronomia',
       'area_Gerencia / Dirección General', 'area_Ingeniería  Industrial',
       'area_Ingeniería  Mecánica', 'area_Ingeniería Electromecánica',
       'area_Ingeniería Química', 'area_Ingeniería de Procesos',
       'area_Ingeniería de Ventas'

In [139]:
todoSinNulos = todoMergeadoFINAL.fillna(0);

In [141]:
todoSinNulos.info

<bound method DataFrame.info of      area_Abastecimiento  area_Administración  \
0                    0.0                    0   
1                    0.0                    0   
2                    0.0                    0   
3                    0.0                    0   
4                    0.0                    0   
5                    0.0                    0   
6                    0.0                    0   
7                    0.0                    0   
8                    0.0                    0   
9                    0.0                    0   
10                   0.0                    0   
11                   0.0                    0   
12                   0.0                    0   
13                   0.0                    0   
14                   0.0                    0   
15                   0.0                    0   
16                   0.0                    0   
17                   0.0                    0   
18                   0.0             

In [167]:
# Load libraries
import pandas
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor

In [159]:
datosForTesting = todoSinNulos.head(20);
datosForTraining = todoSinNulos.tail(120);

In [164]:
datosForTesting.drop('sepostulo',axis=1,inplace=True);

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [170]:
datosForTraining.drop(['idaviso','idpostulante'],axis=1,inplace=True);

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [174]:
datosForTraining.drop(['fechanacimiento','fechapostulacion'],axis=1,inplace=True);

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [181]:
datosForTraining.drop(['descripcion'],axis=1,inplace=True);

ValueError: labels ['descripcion'] not contained in axis

In [185]:
datosForTraining.drop(['titulo'],axis=1,inplace=True);

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [186]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTraining, datosForTraining['sepostulo'], test_size=0.7, random_state=7)

In [187]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 1.000000 (0.000000)
LDA: 0.883333 (0.145297)
KNN: 0.525000 (0.232886)
CART: 0.966667 (0.100000)
NB: 1.000000 (0.000000)
SVM: 0.675000 (0.237024)


C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\nicol\Anacond